In [2]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [53]:
article = pd.read_csv(r'E:\ARAS\customer churn\codes\articles.csv')
df = pd.read_csv(r'E:\ARAS\customer churn\codes\Aras_transactions.csv')
df = pd.merge(df, article, on='articles_id', how='inner')
df

,factor_id,customers_id,price_purchase,art_name,price_item,BranchCode,date,d_dat,articles_id,group_id
0,5,9120623592,220000.0,مگا برگر,83500.0,13813.0,1400-12-21,2022-03-12,1000.0,0
1,5,9120623592,220000.0,پيتزا چهار فصل,103500.0,13813.0,1400-12-21,2022-03-12,1001.0,1
2,50,9397257065,158500.0,پيتزا پپروني,88500.0,13813.0,1400-12-23,2022-03-14,1002.0,1
3,50,9397257065,158500.0,سيب زميني مخصوص,42500.0,13813.0,1400-12-23,2022-03-14,1003.0,2
4,50,9397257065,158500.0,زيرو قوطي,17400.0,13813.0,1400-12-23,2022-03-14,1004.0,3
...,...,...,...,...,...,...,...,...,...,...
3955556,2317955,91309600180,796500.0,سيب زميني مخصوص,177000.0,13812.0,1402-09-29,2023-12-20,1003.0,2
3955557,2317956,93362669140,215000.0,فيله استريپس 4 تيکه,178500.0,13810.0,1401-12-27,2023-03-18,1018.0,9
3955558,2317956,91289353360,443000.0,پيتزا دابل,274500.0,13812.0,1402-09-29,2023-12-20,1029.0,1
3955559,2317956,91289353360,443000.0,سيب زميني مخصوص,88500.0,13812.0,1402-09-29,2023-12-20,1003.0,2


In [54]:
df = df[['customers_id', 'price_purchase', 'd_dat', 'group_id']]
df['date'] = pd.to_datetime(df['d_dat'])
df = df.drop('d_dat', axis = 1)
df

,customers_id,price_purchase,group_id,date
0,9120623592,220000.0,0,2022-03-12
1,9120623592,220000.0,1,2022-03-12
2,9397257065,158500.0,1,2022-03-14
3,9397257065,158500.0,2,2022-03-14
4,9397257065,158500.0,3,2022-03-14
...,...,...,...,...
3955556,91309600180,796500.0,2,2023-12-20
3955557,93362669140,215000.0,9,2023-03-18
3955558,91289353360,443000.0,1,2023-12-20
3955559,91289353360,443000.0,2,2023-12-20


In [55]:
df['day_of_week'] = df['date'].dt.day_name()
df['customers_id'] = df['customers_id'].astype(str)

# Filter rows where 'customer_id' starts with '9' and the length is between 8 and 13
df = df[df['customers_id'].str.startswith('9') & df['customers_id'].str.len().between(8, 13)]
df['customers_id'] = df['customers_id'].astype('int64')

In [56]:
df['purchase_month'] = df['date'].dt.to_period('M')

In [57]:
start_cutoff_date = pd.Timestamp('2023-07-01')
end_cutoff_date = pd.Timestamp('2023-12-01')
month_year = end_cutoff_date.strftime('%Y-%m')

In [58]:
df['customers_id'].nunique()

432280

In [182]:
df_valid = df[(df['date'] >= start_cutoff_date) & (df['date'] <= end_cutoff_date)]

In [183]:
df_valid['customers_id'].nunique()

88912

In [184]:
df_valid = df_valid.drop_duplicates(subset=['customers_id', 'date'], keep='first')

In [185]:
# Step 1: Identify customers to drop based on the original DataFrame
customers_to_drop_train = df_valid['customers_id'].value_counts()[df_valid['customers_id'].value_counts() <= 2].index

# Step 2: Drop these customers
df_valid.drop(df_valid[df_valid['customers_id'].isin(customers_to_drop_train)].index, inplace=True)


# Step 5: Calculate the number of unique customers
unique_customers = df_valid['customers_id'].nunique()
print(unique_customers)


12610


In [187]:
df_valid

,customers_id,price_purchase,group_id,date,day_of_week,purchase_month
242236,9104886255,559500.0,19,2023-07-06,Thursday,2023-07
244617,9123441797,237500.0,9,2023-07-10,Monday,2023-07
246950,9104886255,513500.0,0,2023-07-13,Thursday,2023-07
248662,9126593971,191000.0,3,2023-07-16,Sunday,2023-07
250847,9121583628,162500.0,1,2023-07-22,Saturday,2023-07
...,...,...,...,...,...,...
3955406,91978375960,344500.0,1,2023-11-30,Thursday,2023-11
3955418,91220080120,219500.0,14,2023-11-30,Thursday,2023-11
3955472,99030357430,97500.0,1,2023-11-30,Thursday,2023-11
3955487,91200544130,336000.0,0,2023-11-30,Thursday,2023-11


In [190]:
df_valid['date'].min(), df_valid['date'].max()

(Timestamp('2023-07-01 00:00:00'), Timestamp('2023-12-01 00:00:00'))

In [191]:
import lifetimes

def calculate_metrics(df, current_date):
    metrics_pd = (
        lifetimes.utils.summary_data_from_transaction_data(
            df,
            customer_id_col='customers_id',
            datetime_col='date',
            observation_period_end=current_date, 
            freq='D'
        )
    )
    filtered_pd = metrics_pd[metrics_pd['frequency'] > 0]
    filtered_pd.reset_index(inplace=True)  # Ensure customers_id is a column
    return filtered_pd

current_date = df_valid['date'].max()

metrics_valid = calculate_metrics(df_valid, current_date)

In [192]:
# Checking the number of unique customers
print("Number of unique customers in df_valid:", df_valid['customers_id'].nunique())
print("Number of unique customers in metrics_valid:", metrics_valid['customers_id'].nunique())

Number of unique customers in df_valid: 12610
Number of unique customers in metrics_valid: 12610


In [193]:
from lifetimes import BetaGeoFitter

# Function to fit BG/NBD model and predict expected number of purchases in the next 30 days
def predict_expected_purchases(metrics_pd, t=30, penalizer_coef=0.01):
    bgf = BetaGeoFitter(penalizer_coef=penalizer_coef)
    bgf.fit(metrics_pd['frequency'], metrics_pd['recency'], metrics_pd['T'])
    
    # Predict the expected number of purchases for the next t days
    metrics_pd['expected_purchases_30'] = bgf.predict(t, metrics_pd['frequency'], metrics_pd['recency'], metrics_pd['T'])
    return metrics_pd


# Fit BG/NBD model and predict expected number of purchases in the next 30 days
metrics_valid = predict_expected_purchases(metrics_valid, t=30, penalizer_coef=0.1)

save

In [194]:
metrics_valid[['customers_id', 'expected_purchases_30']].to_csv('metrics_valid.csv', index=False)

In [195]:
metrics_valid['customers_id'].nunique(), df_valid['customers_id'].nunique()

(12610, 12610)